# Практическая работа по дисциплине "Теория принятия решений"
## Тема: Применение методов линейного и динамического программирования для решения практических задач (по вариантам)
### Вариант 47

**Половников Никита**

**группа 1374**

## Задача 3
    
        Мебельный магазин в целях быстрого удовлетворения спроса покупателей может размещать заказы на поставку диванов в начале каждого месяца. Диваны поставляются в магазин автотранспортом, причем за один рейс можно перевезти не более двух диванов. Затраты на поставку диванов складываются из стоимости рейса 60 денежных единиц (ДЕ) и затрат на поставку каждого дивана 120 ДЕ. 
        Торговый зал позволяет разместить не более 7 диванов. Затраты пункта продажи на хранение одного дивана в течение месяца составляют 15 ДЕ. Доход от каждого проданного дивана составляет 600 ДЕ. Распределение вероятностей спроса на ближайшие месяцы приведено в табл. 7. 
    
        1. Определите ожидаемый доход магазина за 6 месяцев при реализации стационарной стратегии, заключающейся в заказе 2 диванов, если на начало очередного месяца остается менее 3 диванов. 
    
        2. Определите оптимальную стратегию размещения заказов на последующие 6 месяцев. 
    
        3. Проведите анализ чувствительности оптимальной стратегии размещения заказов к затратам на хранение дивана в пункте продажи.

**Таблица 7: Распределение вероятностей спроса на диваны**

| **Месяц \Количество** | **0** | **1** | **2** | **3** | **4** |  **5** | 
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| **1** | 0.1 | 0.1 | 0.2 | 0.3 | 0.2 | 0.1 | 
| **2** | 0.1 | 0.1 | 0.3 | 0.3 | 0.1 | 0.1 | 
| **3** | 0.1 | 0.1 | 0.3 | 0.3 | 0.1 | 0.1 | 
| **4** | 0.1 | 0.1 | 0.2 | 0.3 | 0.2 | 0.1 | 
| **5** | 0.1 | 0.1 | 0.1 | 0.3 | 0.3 | 0.1 | 
| **6** | 0.1 | 0.1 | 0.1 | 0.3 | 0.3 | 0.1 | 

### Формализация задачи 

*Решаем задачу минимизации затрат предприятия на отрезке 5 лет.*

Уравнение Беллмана для задачи с неопределенностью:

$ \normalsize W_i(s_j) = max{{\{}\sum{p_{x,i,q}}(\omega(x,s_j,s_q) + W_{i+1}(s_q))\}}$

-  **$\normalsize W_i(s_j)$** -  ожидаемый доход магазина в месяц i, если на начало месяца остается $s_j$ диванов.

- **$\normalsize p_{x,i,q}$** - вероятность того, что спрос составит $x$ диванов в месяц $i$, если на начало месяца остается $s_j$ диванов, и магазин закажет $q$ диванов.

- **$\normalsize \omega(x,s_j,s_q)$** - доход магазина в месяц $i$, если на начало месяца остается $s_j$ диванов, спрос составляет $x$ диванов, а магазин закажет $q$ диванов.

**Целевая функция:**

Необходимо максимизировать ожидаемый доход магазина за 6 месяцев.

**Ограничения:**

- В начале каждого месяца остается не более 7 диванов.
- Заказывается не более 2 диванов в месяц.

In [11]:
import numpy as np

# Распределение вероятностей спроса на диваны
demand_probabilities = np.array([
    [0.1, 0.1, 0.2, 0.3, 0.2, 0.1],
    [0.1, 0.1, 0.3, 0.3, 0.1, 0.1],
    [0.1, 0.1, 0.3, 0.3, 0.1, 0.1],
    [0.1, 0.1, 0.2, 0.3, 0.2, 0.1],
    [0.1, 0.1, 0.1, 0.3, 0.3, 0.1],
    [0.1, 0.1, 0.1, 0.3, 0.3, 0.1]
])

# Затраты на поставку и хранение диванов
delivery_cost_per_sofa = 120
delivery_cost_per_trip = 60
storage_cost_per_sofa_per_month = 15
revenue_per_sofa_sold = 600

# Максимальное количество диванов в магазине
max_sofas_in_store = 7

# Максимальное количество диванов, которое можно заказать за один раз
max_sofas_per_order = 2

# Количество месяцев
num_months = 6

# Инициализация массива для хранения ожидаемого дохода
expected_income = np.zeros((num_months + 1, max_sofas_in_store + 1))

# Установка начального количества диванов в магазине
initial_sofas_in_store = 3

# Решение уравнения Беллмана с учетом стационарной стратегии
for month in range(num_months - 1, -1, -1): # Обратный цикл по месяцам
    for sofas_in_store in range(max_sofas_in_store + 1): # Перебираем количество диванов в магазине
        max_income = float("-inf")
        for order in range(min(max_sofas_per_order, max_sofas_in_store - sofas_in_store) + 1): # Перебираем количество заказываемых диванов
            trips = int(np.ceil(order / 2))  # Определяем количество рейсов
            income = 0
            for sold_sofas, prob in enumerate(demand_probabilities[month]): # Обходим вероятности спроса
                sold_sofas = min(sold_sofas, sofas_in_store + order)  # Учитываем только проданные диваны
                storage_cost = storage_cost_per_sofa_per_month * (sofas_in_store + order - sold_sofas)  # Затраты на хранение (1 замечание)
                income += round(prob * (sold_sofas * revenue_per_sofa_sold +  # Учитываем доход и затраты
                        order * delivery_cost_per_sofa - 
                        trips * delivery_cost_per_trip -  
                        storage_cost+                                         # Вычитаем затраты на хранение диванов, оставшихся в магазине
                        expected_income[month + 1][min(max_sofas_in_store, sofas_in_store + order - sold_sofas)])) # W[i+1](2 замечание)
            income -= int(storage_cost_per_sofa_per_month * (sofas_in_store + order - sold_sofas))
            max_income = max(max_income, income) # Находим максимальный доход
        expected_income[month][sofas_in_store] = max_income

# Ожидаемый доход за 6 месяцев
total_income = expected_income[0][initial_sofas_in_store]

print("Ожидаемый доход магазина за 6 месяцев при реализации стационарной стратегии:", total_income)


Ожидаемый доход магазина за 6 месяцев при реализации стационарной стратегии: 9266.0


1. Да, действительно спрос ограничен **sofas_in_store + order**.

2. Исправил цикл **for sold_sofas** для соответствия уравнению Беллмана: $\newline$ $\bullet$ "Затраты на хранение на вероятность не домножаются": $\newline$ Добавил переменную **storage_cost = storage_cost_per_sofa_per_month * (sofas_in_store + order - sold_sofas)**, которая умножается на вероятности. $\newline$ $\bullet$ "Нигде не добавляется W_{i+1}" : $\newline$ 
$W_{i+1}(s_q))$ Это ожидаемый доход в следующем месяце $i+1$, если в начале месяца в магазине останется $s_q$ диванов. $\newline$ **expected_income[month + 1][min(max_sofas_in_store, sofas_in_store + order - sold_sofas)]** - это ожидаемый доход в следующем месяце $i+1$, если в начале месяца будет **min(max_sofas_in_store, sofas_in_store + order - sold_sofas)** диванов.